In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader=WebBaseLoader("https://docs.smith.langchain.com/observability/tutorials/observability")
loader

In [13]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='\n\n\n\n\nAdd observability to your LLM application | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityTutorialsAdd observability to your LLM applicationHow-to GuidesTracingMonitoring and automationsConceptual GuideEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceObservabilityTutorialsAdd observability to your 

In [14]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [15]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityTutorialsAdd observability to your LLM applicationHow-to GuidesTracingMonitoring and automationsCo

In [16]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [17]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [10]:
vectorstoredb

In [18]:
query="The next stage of LLM application development is beta testing your application"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Beta Testing\u200b\nThe next stage of LLM application development is beta testing your application.\nThis is when you release it to a few initial users.\nHaving good observability set up here is crucial as often you don't know exactly how users will actually use your application, so this allows you get insights into how they do so.\nThis also means that you probably want to make some changes to your tracing set up to better allow for that.\nThis extends the observability you set up in the previous section\nCollecting Feedback\u200b\nA huge part of having good observability during beta testing is collecting feedback.\nWhat feedback you collect is often application specific - but at the very least a simple thumbs up/down is a good start.\nAfter logging that feedback, you need to be able to easily associate it with the run that caused that.\nLuckily LangSmith makes it easy to do that.\nFirst, you need to log the feedback from your app."

In [19]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4-0125-preview")

In [20]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x77f654187010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x77f654379120>, root_client=<openai.OpenAI object at 0x77f6543c2710>, root_async_client=<openai.AsyncOpenAI object at 0x77f654187040>, model_name='gpt-4-0125-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [21]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x77f654187010>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x77f654379120>, root_client=<openai.OpenAI object at 0x77f6543c2710>, root_async_client=<openai.AsyncOpenAI object at 0x77f654187040>, model_name='gpt-4-0125-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [22]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"The next stage of LLM application development is beta testing your application",
    "context":[Document(page_content="The next stage of LLM application development is beta testing your application. This is when you release it to a few initial users.")]
})

'Beta testing your application involves releasing it to a select group of initial users to collect feedback on its functionality, usability, and potential improvements before a wider release. This phase is critical for identifying and fixing any issues to enhance the user experience.'

In [23]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [24]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [25]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x77f670da09d0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | Chat

In [27]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"The next stage of LLM application development is beta testing your application"})
response['answer']

'Based on the provided context, the discussion is centered on the importance of incorporating observability into the lifecycle of Language Model (LLM) application development, with a specific emphasis on beta testing phases. Observability plays a crucial role in understanding how users interact with an LLM application, given the non-deterministic nature of these models. The tutorial underscores the need to adjust observability measures, like tracing setups, to gain insights into user behavior and collect feedback efficiently. Tools like LangSmith are highlighted for their utility in facilitating this process by offering LLM-native observability features. This ensures that developers can monitor and iterate on their applications effectively, maintaining a feedback loop that informs improvements during beta testing and beyond. Observability is presented as an indispensable element across all stages of development, from prototyping to production, to ensure the delivery of robust LLM appli

In [28]:
response

{'input': 'The next stage of LLM application development is beta testing your application',
 'context': [Document(id='cc358e9c-a139-4ad9-bb03-8c31472efda2', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Beta Testing\u200b\nThe next stage of LLM application development is beta testing your application.\nThis is when you release it to a few initial users.\nHaving good observability set up here is crucial as often you don't know exactly how users will actually use your application, so this allows you get insights into how they do so.\nThis also means that you probably want to make some changes to your tracing set up to better allow for that.\nThis extends the observability you set up in the previous section\nCollecting Feedbac

In [29]:
response['context']

[Document(id='cc358e9c-a139-4ad9-bb03-8c31472efda2', metadata={'source': 'https://docs.smith.langchain.com/observability/tutorials/observability', 'title': 'Add observability to your LLM application | 🦜️🛠️ LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="Beta Testing\u200b\nThe next stage of LLM application development is beta testing your application.\nThis is when you release it to a few initial users.\nHaving good observability set up here is crucial as often you don't know exactly how users will actually use your application, so this allows you get insights into how they do so.\nThis also means that you probably want to make some changes to your tracing set up to better allow for that.\nThis extends the observability you set up in the previous section\nCollecting Feedback\u200b\nA huge part of having good observability during beta testing is collecting feedback.\nWhat feed